# Feature Extraction

In [1]:
import pandas as pd
import re
from sklearn import preprocessing


def cleanData(text):
    removeList = ["@handle","@handle:"]
    text_r = " ".join([char for char in text.split(' ') if char not in removeList]) 
    return text_r

def removeHyperLink(text):
    text,count = re.subn(r'http?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    return(text,count)
    
def removeSpChar(text):
    new = re.sub('[@_!#$%^&*()<>?/\|}{~:\']' ,'', text)
    count = len(text) - len(new)
    return new,count

##Normalization


from sklearn import preprocessing

def normalizeColumn(df_orig,column):
    min_max_scaler = preprocessing.MinMaxScaler()
    x = df_orig[[column]].values #returns a numpy array
    x_scaled = min_max_scaler.fit_transform(x)
    df_orig[[column]] = pd.DataFrame(x_scaled)
    return df_orig


df_orig = pd.read_table('data/train_tweets.txt',names=('id','rtweet'))

df_orig['tweet'] = df_orig['rtweet'].apply(lambda x: cleanData(x))


/Users/saransh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.


In [2]:

df_orig['tweet'],df_orig['NumLink'] = zip(*df_orig['tweet'].map(removeHyperLink))
df_orig['hashtags'] = df_orig['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
df_orig['tweet'],df_orig['spCount'] =  zip(*df_orig['tweet'].map(removeSpChar))

df_orig['wdCount'] = df_orig['tweet'].apply(lambda x: len(str(x).split(" ")))
df_orig['chCount'] = df_orig['tweet'].str.len()
df_orig['isCap'] = df_orig['tweet'].apply(lambda x: len([x for x in x.split() if x[0].isupper()]))
df_orig['tweet'] = df_orig['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_orig['isRT'] = df_orig['tweet'].apply(lambda x: 1 if 'rt' in x else 0)


df_orig

,id,rtweet,tweet,NumLink,hashtags,spCount,wdCount,chCount,isCap,isRT
0,8746,@handle Let's try and catch up live next week!,lets try and catch up live next week,0,0,2,8,36,1,0
1,8746,Going to watch Grey's on the big screen - Thur...,going to watch greys on the big screen - thurs...,0,0,1,11,65,3,0
2,8746,@handle My pleasure Patrick....hope you are well!,my pleasure patrick....hope you are well,0,0,1,6,40,2,0
3,8746,@handle Hi there! Been traveling a lot and lot...,hi there been traveling a lot and lots more to...,0,0,3,26,121,3,0
4,8746,RT @handle Looking to Drink Clean & Go Green? ...,rt looking to drink clean go green purchase yo...,0,0,3,17,90,8,1
5,8746,RT @handle: Ft. Hood officials confirm the 2 o...,rt ft. hood officials confirm the 2 other sold...,0,0,0,16,96,3,1
6,8746,RT @handle: Mickey Mouse is Getting a Make Ove...,rt mickey mouse is getting a make over,1,0,0,9,39,6,1
7,8746,@handle How did u get the invite Justin?,how did u get the invite justin,0,0,1,7,31,2,0
8,8746,@handle I think I am still a good friend of he...,i think i am still a good friend of hers - lol,0,0,1,12,46,2,0
9,8746,@handle I remember! I am fine - how are u? Wha...,i remember i am fine - how are u whats new,0,0,4,11,42,3,0


In [3]:

normalizeColumn(df_orig,'chCount')
normalizeColumn(df_orig,'wdCount')
normalizeColumn(df_orig,'spCount')
normalizeColumn(df_orig,'isCap')
normalizeColumn(df_orig,'hashtags')



/Users/saransh/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/saransh/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/saransh/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/saransh/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/saransh/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning:

,id,rtweet,tweet,NumLink,hashtags,spCount,wdCount,chCount,isCap,isRT
0,8746,@handle Let's try and catch up live next week!,lets try and catch up live next week,0,0.000000,0.015625,0.007830,0.005966,0.003436,0
1,8746,Going to watch Grey's on the big screen - Thur...,going to watch greys on the big screen - thurs...,0,0.000000,0.007812,0.011186,0.010772,0.010309,0
2,8746,@handle My pleasure Patrick....hope you are well!,my pleasure patrick....hope you are well,0,0.000000,0.007812,0.005593,0.006629,0.006873,0
3,8746,@handle Hi there! Been traveling a lot and lot...,hi there been traveling a lot and lots more to...,0,0.000000,0.023438,0.027964,0.020053,0.010309,0
4,8746,RT @handle Looking to Drink Clean & Go Green? ...,rt looking to drink clean go green purchase yo...,0,0.000000,0.023438,0.017897,0.014915,0.027491,1
5,8746,RT @handle: Ft. Hood officials confirm the 2 o...,rt ft. hood officials confirm the 2 other sold...,0,0.000000,0.000000,0.016779,0.015910,0.010309,1
6,8746,RT @handle: Mickey Mouse is Getting a Make Ove...,rt mickey mouse is getting a make over,1,0.000000,0.000000,0.008949,0.006463,0.020619,1
7,8746,@handle How did u get the invite Justin?,how did u get the invite justin,0,0.000000,0.007812,0.006711,0.005138,0.006873,0
8,8746,@handle I think I am still a good friend of he...,i think i am still a good friend of hers - lol,0,0.000000,0.007812,0.012304,0.007623,0.006873,0
9,8746,@handle I remember! I am fine - how are u? Wha...,i remember i am fine - how are u whats new,0,0.000000,0.031250,0.011186,0.006961,0.010309,0


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(df_orig['tweet'])
X = vectorizer.transform(df_orig['tweet'])



In [7]:
df2 = df_orig.drop(columns=["rtweet","tweet"])
X.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [14]:
X[1]

<1x137941 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [ ]:
count_vect_df = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())

combined_df = pd.concat([df2, count_vect_df], axis=1)


#combined_df = pd.concat([df_orig.frop, df_orig['tweetBoW]], axis=1)
combined_df

# Model Training

In [17]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

dataset = df_orig.drop(columns=["rtweet","tweet"])
#print(dataset)
#names = ['authorID','NumLink','hastags','spCount','word_count','char_count','isCapitalize','RT_count']
X = dataset.iloc[:, 1:].values
y = dataset.iloc[:,0].values
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
#dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y=encoded_Y
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.30,random_state=42)
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu')) # input dimension = dimension of festure vector
model.add(Dense(6, activation='relu'))
model.add(Dense(10000, activation='softmax')) # output layer = no. of classes
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
train_histt = model.fit(X_train, y_train, epochs=2, batch_size=256, verbose=1)






Epoch 1/2


ValueError: setting an array element with a sequence.